# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.31.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacity-p3
udacity-project-3
eastus2
57edfb84-e1fe-4fdf-8da0-d05684406ce1


In [3]:
experiment_name = 'telco-churn-hyperparameter'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
telco-churn-hyperparameter,udacity-p3,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = 'cpucluster'

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found Existing Cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found Existing Cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
ws = Workspace.from_config()
experiment_name = 'telco-churn-hyperparameter'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [24]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=10, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
                    {
                        "--C": choice(10, 20,30,40, 50,60,70,80,90, 100, 200, 300,500),
                        '--max_iter': choice(0.1,0.5,1,2,5,10)
                    }
                )

#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory = "./",
                compute_target=cpu_cluster,
                vm_size='STANDARD_D2_v2',
                entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal =PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4
                                    )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7fe7f806-5d43-4869-ac19-1685c2fc1842
Web View: https://ml.azure.com/runs/HD_7fe7f806-5d43-4869-ac19-1685c2fc1842?wsid=/subscriptions/57edfb84-e1fe-4fdf-8da0-d05684406ce1/resourcegroups/udacity-project-3/workspaces/udacity-p3&tid=69c15541-01ca-426a-a860-b0b1c1a0e229

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-07-27T22:27:59.478711][API][INFO]Experiment created<END>\n""<START>[2021-07-27T22:27:59.895576][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-07-27T22:28:00.059041][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_7fe7f806-5d43-4869-ac19-1685c2fc1842
Web View: https://ml.azure.com/runs/HD_7fe7f806-5d43-4869-ac19-1685c2fc1842?wsid=/subscriptions/57edfb84-e1fe-4fdf-8da0-d05684406ce1/resourcegroups/udacity-project-3/workspaces/udacity-p3&tid=69c15541-01ca-426a-a860-b0b1c1a0e229

Warnings:
{
  "error": {
    "code": "UserError",
    

{'runId': 'HD_7fe7f806-5d43-4869-ac19-1685c2fc1842',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-27T22:27:59.248319Z',
 'endTimeUtc': '2021-07-27T22:58:09.760781Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [26]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_run.get_children_sorted_by_primary_metric(top=1)

[{'run_id': 'HD_7fe7f806-5d43-4869-ac19-1685c2fc1842_77',
  'hyperparameters': '{"--C": 100, "--max_iter": 5}',
  'best_primary_metric': 0.7746979388770433,
  'status': 'Completed'}]

In [27]:
best_run.get_details()

{'runId': 'HD_7fe7f806-5d43-4869-ac19-1685c2fc1842_2',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-27T22:33:42.070706Z',
 'endTimeUtc': '2021-07-27T22:34:39.880338Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'e3a9f6d4-2b57-467b-9b0e-4b8ec778fca6',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.RuntimeType': 'Hosttools'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '20', '--max_iter', '10'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpucluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environ

In [28]:
best_run.get_metrics()

{'Regularization Strength:': 20.0,
 'Max iterations:': 10,
 'Accuracy': 0.7746979388770433}

In [29]:
print("Best Hyperdrive Run ID: " + best_run.id)

Best Hyperdrive Run ID: HD_7fe7f806-5d43-4869-ac19-1685c2fc1842_2


In [32]:
#TODO: Save the best model
model = best_run.register_model(model_name='best_run_hyperdrive.pkl', model_path ='./model_outputs/')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path model_outputs in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_3ecf88aded899f44ec0d21aca345c06f9ef2930f97daf501930668dd0133c49b_d.txt', 'azureml-logs/65_job_prep-tvmps_3ecf88aded899f44ec0d21aca345c06f9ef2930f97daf501930668dd0133c49b_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_3ecf88aded899f44ec0d21aca345c06f9ef2930f97daf501930668dd0133c49b_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/108_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path model_outputs in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_3ecf88aded899f44ec0d21a

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service